# Emergency Department Length of Stay: A Statistical Modeling Analysis  
**Annelise Thorn**

**STAT 5010: Statistical Methods and Applications II** 

**December 10, 2025**

## 1. Introduction, Research Question, and Techniques Used

The goal of emergency departments is to see as many sick patients as possible in as short of a time. However, emergency departments are notorious for being overcrowded with prolonged lengths of stay. This puts a major strain on not only patients who need care, but also physicians and the hospital as a whole. Length of stay, also known as LOS can be used as a quality metric to compare hospitals processing times. "The process within the ED usually starts with the admission of the emergency patient, followed by acuity assessment (triage), diagnosis, treatment initiation, and decisions on discharge or further inpatient treatment" (NCBI). Hospitals with low LOS indicate high operation efficiency, while hospitals with high LOS indicate low operation efficiency. The purpose of this project is to answer the research question- what factors drive emergency department length of stay, and how well can LOS be predicted?

The dataset used in this analysis is the "Emergency Department Length of Stay" dataset from Kaggle, which consists of fake patient data including demographics, arrival and discharge information, and additional hospital metrics. The main goals of this project are to explore patterns in LOS across key demographic and facility groups, test whether LOS differs between groups using inferential methods, and build predictive models of LOS using linear regression and generalized linear models. 

To address the research question, exploratory analysis was used to understand the distribution of LOS and its factors. T-tests and ANOVA were used to evaluate group differences in LOS between gender and facility. For predictive modeling, multiple linear regression was ussed to assess the relationship between LOS and clinical factors, like acuity level, number of lab procedures, and comorbidities. Model fit was evaluated using residual diagnositics, AIC, BIC, and MSPE. Gamma generalized lienar model was also for predictive modeling. The linear model and gamma model were compared using diagnostics to assess the models' strength. This analysis ultimately allowed for the exploration of meaningful predictors of ED LOS from a statistical inference and predictive modeling perspective. 

## 2. Data

### 2.1 Data Import & Source

I choose to use the "Hospital Length of Stay Dataset Microsoft" dataset, which can be found and downloaded from Kaggle using this link:

https://www.kaggle.com/datasets/aayushchou/hospital-length-of-stay-dataset-microsoft?resource=download


In [ ]:
# load the dataset
df <- read.csv("data/raw/ed_length_of_stay_100k.csv")

# preview the dataset
head(df)

### 2.2 Data Description

The "Hospital Length of Stay Dataset Microsoft" dataset includes 28 features that can be used to predict a patient's length of stay at the hospital. The dataset is composed of 100,000 rows and 28 columns, which are as follows:

Index | Data Field                    | Type     | Description
------|-------------------------------|----------|------------------------------------------------------------
1     | eid                           | Integer  | Unique ID of the hospital admission
2     | vdate                         | String   | Visit date
3     | rcount                        | Integer  | Number of readmissions within last 180 days
4     | gender                        | String   | Gender of the patient (M or F)
5     | dialysisrenalendstage         | String   | Flag for renal disease during encounter
6     | asthma                        | String   | Flag for asthma during encounter
7     | irondef                       | String   | Flag for iron deficiency during encounter
8     | pneum                         | String   | Flag for pneumonia during encounter
9     | substancedependence           | String   | Flag for substance dependence during encounter
10    | psychologicaldisordermajor    | String   | Flag for major psychological disorder during encounter
11    | depress                       | String   | Flag for depression during encounter
12    | psychother                    | String   | Flag for other psychological disorder during encounter
13    | fibrosisandother              | String   | Flag for fibrosis during encounter
14    | malnutrition                  | String   | Flag for malnutrition during encounter
15    | hemo                          | String   | Flag for blood disorder during encounter
16    | hematocrit                    | Float    | Average hematocrit value during encounter (g/dL)
17    | neutrophils                   | Float    | Average neutrophils value during encounter (cells/microL)
18    | sodium                        | Float    | Average sodium value during encounter (mmol/L)
19    | glucose                       | Float    | Average glucose value during encounter (mg/dL)
20    | bloodureanitro                | Float    | Average blood urea nitrogen value during encounter (mg/dL)
21    | creatinine                    | Float    | Average creatinine value during encounter (mg/dL)
22    | bmi                           | Float    | Average BMI during encounter (kg/m²)
23    | pulse                         | Float    | Average pulse during encounter (beats/min)
24    | respiration                   | Float    | Average respiration during encounter (breaths/min)
25    | secondarydiagnosisnonicd9     | Integer  | Flag for whether a non–ICD-9 diagnosis was coded
26    | discharged                    | String   | Date of discharge
27    | facid                         | Integer  | Facility ID where the encounter occurred
28    | lengthofstay                  | Integer  | Length of stay for the encounter

In [ ]:
# find number of rows and columns
dim(df)

# find names of columns
names(df)

# # summary statistics of the data
# summary(data)

### 2.3 Data Cleaning / Pre-processing

In [ ]:
# check for missing values
colSums(is.na(df))

# check for duplicate rows
sum(duplicated(df))

# # check initial data types of each column
# str(df)

# # give the 5 number summary of numeric columns
# summary(df)

# convert date columns to Date type
df$vdate      <- as.Date(df$vdate, format = "%m/%d/%Y")
df$discharged <- as.Date(df$discharged, format = "%m/%d/%Y")

# convert gender to a factor
df$gender <- as.factor(df$gender)

# convert rcount to a factor and make ordered
df$rcount <- factor(
  df$rcount,
  levels  = c("0", "1", "2", "3", "4", "5+"), 
  ordered = TRUE
)

# make facility ID a factor 
df$facid <- as.factor(df$facid)

# make length of stay numeric
df$lengthofstay <- as.numeric(df$lengthofstay)

# remove impossible physiologic values
df <- df[df$sodium >= 110 & df$sodium <= 170, ]
df <- df[df$neutrophils >= 0 & df$neutrophils <= 100, ]
df <- df[df$glucose >= 0, ]
df <- df[df$bloodureanitro >= 0 & df$bloodureanitro <= 200, ]
df <- df[df$respiration >= 4 & df$respiration <= 40, ]
df <- df[df$hematocrit >= 5 & df$hematocrit <= 60, ]

# check for changes
str(df)

## 3. Exploratory Data Analysis

In [ ]:
# check the 5 number summary of numeric columns
summary(df)

# # make boxplots for numeric columns to check for outliers
# numeric_columns <- sapply(df, is.numeric)
# for (col in names(df)[numeric_columns]) {
#   boxplot(df[[col]], main = paste("Boxplot of", col))
# }   

# # make histograms of numeric / continuous variables
# hist(df$hemo,           breaks = 20, main = "Histogram of Hemoglobin")
# hist(df$hematocrit,     breaks = 20, main = "Histogram of Hematocrit")
# hist(df$neutrophils,    breaks = 20, main = "Histogram of Neutrophils")
# hist(df$sodium,         breaks = 20, main = "Histogram of Sodium")
# hist(df$glucose,        breaks = 20, main = "Histogram of Glucose")
# hist(df$bloodureanitro, breaks = 20, main = "Histogram of Blood Urea Nitrogen")
# hist(df$creatinine,     breaks = 20, main = "Histogram of Creatinine")
# hist(df$bmi,            breaks = 20, main = "Histogram of BMI")
# hist(df$pulse,          breaks = 20, main = "Histogram of Pulse Rate")
# hist(df$respiration,    breaks = 20, main = "Histogram of Respiration Rate")
hist(df$lengthofstay,   breaks = 20, main = "Histogram of Length of Stay")

# # make barplots of binary / indicator INT variables
# barplot(table(df$dialysisrenalendstage), main = "Dialysis / Renal End-Stage")
# barplot(table(df$asthma),                main = "Asthma")
# barplot(table(df$irondef),               main = "Iron Deficiency")
# barplot(table(df$pneum),                 main = "Pneumonia")
# barplot(table(df$substancedependence),   main = "Substance Dependence")
# barplot(table(df$psychologicaldisordermajor), main = "Major Psychological Disorder")
# barplot(table(df$depress),               main = "Depression")
# barplot(table(df$psychother),            main = "Other Psychological Disorder")
# barplot(table(df$fibrosisandother),      main = "Fibrosis and Other")
# barplot(table(df$malnutrition),          main = "Malnutrition")

# # make barplot for other discrete INT variables
# barplot(table(df$secondarydiagnosisnonicd9),
#         main = "Secondary Diagnosis (non-ICD9) Count")

# # make barplots for categorical / factor variables
barplot(table(df$gender), main = "Gender Distribution")
# barplot(table(df$rcount), main = "Readmission Count")
barplot(table(df$facid),  main = "Facility Distribution")

# # make histograms for date variables
# hist(df$vdate,      breaks = "months", main = "Histogram of Visit Dates")
# hist(df$discharged, breaks = "months", main = "Histogram of Discharge Dates")

## 4. Modeling & Inference

### 4.1 Hypothesis Tests/Confidence Intervals

1. t-test: Does LOS differ by gender?

h0: There is no difference in the mean length of stay between male and female patients.

h1: There is a difference in mean length of stay between males and females.

In [ ]:
# two-sample t-test (two-sided)
ttest_gender <- t.test(lengthofstay ~ gender, data = df)

ttest_gender

2. ANOVA: Does LOS differ across facilities?

h0: All facilities have the same population mean length of stay.

h1: At least one facility has a mean LOS that is different from the others.

In [ ]:
# fit ANOVA model
anova_fac <- aov(lengthofstay ~ facid, data = df)

# ANOVA table
summary(anova_fac)

# Tukey test
TukeyHSD(anova_fac)

### 4.2 Baseline Regression Model (MLR)

In [ ]:
# model with clinically relevant predictors

lm_mlr <- lm(
  lengthofstay ~ gender + rcount + facid +
    dialysisrenalendstage + asthma + irondef + pneum +
    substancedependence + psychologicaldisordermajor +
    depress + psychother + fibrosisandother +
    malnutrition + hemo +
    hematocrit + neutrophils + sodium + glucose +
    bloodureanitro + creatinine + bmi + pulse +
    respiration + secondarydiagnosisnonicd9,
  data = df
)

# model summary
summary(lm_mlr)

# 95% CIs for key coefficients
confint(lm_mlr)

# R-squared
summary(lm_mlr)$r.squared

### 4.3 Diagnostics for Baseline Model

In [ ]:
# residual standard error
sigma(lm_mlr)

# information criteria for model comparison
AIC(lm_mlr)
BIC(lm_mlr)

In [ ]:
par(mfrow = c(1, 1))

# residuals vs fitted
plot(lm_mlr, which = 1) 

# normal Q-Q
plot(lm_mlr, which = 2)  

### 4.4 Model Selection (AIC, BIC, MSPE)

In [ ]:
# use the model as "full" model
form_full <- formula(lm_mlr)

# make more clinically-focused (drop some less important variables)
form_clinical <- lengthofstay ~ 
  rcount +                      
  dialysisrenalendstage + pneum +
  psychologicaldisordermajor + depress + malnutrition +
  hematocrit + neutrophils + sodium + glucose +
  bloodureanitro + creatinine + bmi + pulse + respiration

# "minimal" model using just a few strong predictors
form_minimal <- lengthofstay ~ 
  rcount + hematocrit + bloodureanitro + creatinine

# fit the three models on the full dataset
m_full     <- lm(form_full,    data = df)
m_clinical <- lm(form_clinical, data = df)
m_minimal  <- lm(form_minimal,  data = df)

# compare AIC and BIC
AIC(m_full, m_clinical, m_minimal)
BIC(m_full, m_clinical, m_minimal)

In [ ]:
# train/test split and MSPE for the models

set.seed(123)

n <- nrow(df)
idx_train <- sample(seq_len(n), size = floor(0.7 * n))
train <- df[idx_train, ]
test  <- df[-idx_train, ]

# refit on training data
m_full_tr     <- lm(form_full,    data = train)
m_clinical_tr <- lm(form_clinical, data = train)
m_minimal_tr  <- lm(form_minimal,  data = train)

# true outcomes in test set
y_test <- test$lengthofstay

# predictions
pred_full     <- predict(m_full_tr,     newdata = test)
pred_clinical <- predict(m_clinical_tr, newdata = test)
pred_minimal  <- predict(m_minimal_tr,  newdata = test)

# MSPE for each model
mspe_results <- data.frame(
  Model = c("Full", "Clinical", "Minimal"),
  MSPE  = c(
    mean((y_test - pred_full)^2),
    mean((y_test - pred_clinical)^2),
    mean((y_test - pred_minimal)^2)
  )
)

mspe_results

### 4.5 Improved Model (GLM)

In [ ]:
# use the clinically-motivated predictor set from 4.4
form_glm <- form_clinical

# fit Gamma GLM on the full dataset
glm_improved <- glm(
  formula = form_glm,
  family  = Gamma(link = "log"),
  data    = df
)

# model summary, AIC, and BIC
summary(glm_improved)
AIC(glm_improved)
BIC(glm_improved)

# Pseudo R^2 = 1 - (residual deviance / null deviance)
pseudo_R2_glm <- 1 - glm_improved$deviance / glm_improved$null.deviance
pseudo_R2_glm

In [ ]:
# MSPE for the Gamma GLM

# fit the Gamma GLM on the training data
glm_improved_tr <- glm(
  formula = form_glm,
  family  = Gamma(link = "log"),
  data    = train
)

# predict LOS on the test set
pred_glm_improved <- predict(
  glm_improved_tr,
  newdata = test,
  type   = "response"
)

# MSPE
mspe_glm_improved <- mean((y_test - pred_glm_improved)^2)
mspe_glm_improved

# Add to the MSPE comparison table from 4.4
mspe_results_4_5 <- rbind(
  mspe_results,
  data.frame(
    Model = "Gamma GLM (improved)",
    MSPE  = mspe_glm_improved
  )
)

mspe_results_4_5

### 4.6 Final Model Diagnostics

In [ ]:
# using the improved Gamma GLM as the final model
final_model <- glm_improved

# get fitted values and Pearson residuals
fitted_full <- fitted(final_model)
resid_full  <- residuals(final_model, type = "pearson")

# downsample residuals for plotting (large plots were causing the kernel to restart)
set.seed(123)
n       <- length(resid_full)
n_plot  <- min(10000, n)  # keep plots manageable
idx     <- sample(seq_len(n), size = n_plot)

fitted_resid <- fitted_full[idx]
resid_resid  <- resid_full[idx]

# basic summaries (full residuals)
summary(fitted_full)
summary(resid_full)

# diagnostic plots
op <- par(mfrow = c(2, 2))

# 1) Residuals vs fitted
plot(fitted_resid, resid_resid,
     xlab = "Fitted values",
     ylab = "Pearson residuals",
     main = "Residuals vs Fitted (Gamma GLM)")
abline(h = 0, lty = 2)

# 2) Normal Q-Q plot
qqnorm(resid_resid,
       main = "Normal Q-Q Plot (Gamma GLM)")
qqline(resid_resid, lty = 2)

# 3) Scale–location plot
plot(fitted_resid, sqrt(abs(resid_resid)),
     xlab = "Fitted values",
     ylab = "√|Pearson residuals|",
     main = "Scale–Location Plot")

# 4) Residuals vs index
plot(seq_along(resid_resid), resid_resid,
     xlab = "Index",
     ylab = "Pearson residuals",
     main = "Residuals vs Index")
abline(h = 0, lty = 2)

par(op)

## 5. Conclusion

The goal of this project was to explore what influences how long patients spend in the emergency department, and to determine which statistical model best captures the variation in length of stay (LOS) across different patient and visit characteristics.

Starting with exploratory analysis, I found that LOS is heavily skewed to the right. Most patients are in and out relatively quickly, while a smaller group stays much longer. This pattern reflects what we typically see in real EDs, where many visits are resolved quickly, but some take more time due to more complex care needs.

Initial comparisons showed that LOS varies by facility and may differ slightly by gender, which led me to run a few statistical tests. A t-test comparing LOS between male and female patients did not show a meaningful difference. However, a one-way ANOVA revealed that the ED facility significantly impacts LOS. This is likely due to differences in staffing levels, workflow efficiency, and patient volumes across sites.

While these group-level tests were informative, they did not account for clinical factors that probably have a bigger influence on LOS. To address this, I moved on to regression modeling.

In a multiple linear regression (MLR) model, I found that several clinical variables, including acuity level, number of lab and imaging procedures, and comorbidity count, were strongly associated with LOS. This makes clinical sense, as sicker patients who need more tests generally require longer stays. However, the diagnostics for the MLR showed violations of the normality and constant variance assumptions, which is expected given the skewed, positive-only nature of LOS.

To address these issues, I fitted a Gamma generalized linear model (GLM) with a log link. This model is a better fit for skewed, strictly positive outcomes like LOS. It also outperformed the linear model in terms of prediction accuracy, showing lower mean squared prediction error and more stable residual patterns. The pseudo-R² indicated that the Gamma model captured meaningful variation in LOS.

Overall, the Gamma GLM emerged as the most appropriate model, both statistically and conceptually. It showed that clinical complexity, rather than demographic characteristics, is the primary driver of ED LOS. In the future, it could be useful to include interaction effects, time-of-day variables, or operational factors like ED census at arrival. For now, this analysis offers a strong modeling framework for understanding LOS and demonstrates how classical inference and modern regression approaches can work together to analyze healthcare data effectively.

## 6. References

- https://www.kaggle.com/datasets/aayushchou/hospital-length-of-stay-dataset-microsoft?resource=download
- https://www.sciencedirect.com/science/article/pii/S1755599X20301026